## specifically: stride time (cadence and stride length should be redundant)

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors.v2021 import register_cmaps

from stressgait_analysis.dataset import StressGaitDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2
from stressgait_analysis.gait_helper_functions import compute_HS
import numpy as np
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])

base_path

In [ ]:
dataset = StressGaitDataset(base_path, coarse_condition=True, specify_bouts=True, specify_speed=True)

## computation

In [ ]:
data = []
for trial in dataset:

    keypoints = trial.load_keypoint_trajectories()
    hs_events = compute_HS(keypoints)
    hs_events = [i for i in hs_events if i > 30]
    stride_times = np.diff(hs_events)
    data.extend([
        (*trial.index.values[0], i, st)
        for i, st in zip(range(len(stride_times)), stride_times)
    ])

## store as multilevel df

In [ ]:
stride_df = pd.DataFrame(data, columns=[*dataset.index.columns, "stride_idx", "stride_time"])
stride_df.set_index([*dataset.index.columns, "stride_idx"], inplace=True)
filename = dataset.base_path.joinpath("kinematics/stride_times.csv")
stride_df.to_csv(filename)

In [ ]:
mean_stride_time = stride_df.groupby(["participant", "condition"]).mean()
stride_time_variability = stride_df.groupby(["participant", "condition"]).std()
mean_stride_time

In [ ]:
mean = mean_stride_time.groupby(["condition"]).mean()
mean


# Great! Stride times can now be loaded directly from the dataset!
